# Batı Karadeniz Orman Değişim ve Trend Analizi (2020-2025)

**Amaç:** 2020-2025 yılları arasında Karabük, Bartın ve Zonguldak illerindeki orman değişimlerini izlemek ve değişim nedenlerini (Yangın, Kesim, Maden) ayrıştırmak.

## Yöntem
*   **Veri:** Sentinel-2 Level-2A
*   **Analiz:** Mann-Kendall Trend Testi, Sen's Slope
*   **İndeksler:** NBR, NDVI

In [1]:
import ee
from gee.utils import ee_init
from gee.aoi import get_aoi

# GEE Başlat
try:
    ee_init(project='tubitak-478716') # Proje ID'nizi gerekirse güncelleyin
    print("GEE Initialized")
except Exception as e:
    print(f"Init Error: {e}")

Init Error: Earth Engine init failed for project 'tubitak-478716': Caller does not have required permission to use project tubitak-478716. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=tubitak-478716 and then retry. Propagation of the new permission may take a few minutes.


In [2]:
# 1. Çalışma Alanı: Batı Karadeniz
roi = get_aoi("WESTERN_BLACK_SEA")
print("ROI Info:", roi.getInfo())

EEException: Caller does not have required permission to use project tubitak-478716. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=tubitak-478716 and then retry. Propagation of the new permission may take a few minutes.

## 2. Veri Seti Hazırlığı (2020-2025)
Yıllık veya mevsimsel kompozitler oluşturularak zaman serisi analizi yapılacaktır.

In [ ]:
def get_annual_composite(year, roi):
    """Belirtilen yıl için en yeşil (greenest) piksel kompoziti oluştur."""
    start = f"{year}-06-01"
    end = f"{year}-09-01"
    
    collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterBounds(roi)
                  .filterDate(start, end)
                  .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
                  .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
                  .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR'))
                 )
    
    # Median veya Quality Mosaic kullanılabilir
    return collection.median().clip(roi).set('year', year)

years = range(2020, 2026)
images = [get_annual_composite(y, roi) for y in years]
time_series = ee.ImageCollection(images)
print(f"{len(years)} yıllık seri oluşturuldu.")

## 3. Trend Analizi (Mann-Kendall & Sen's Slope)
GEE üzerinde zaman serisi eğilimini hesaplamak için Sen's Slope reducer'ı ve Mann-Kendall istatistiği kullanılır.
*Not: Bu işlem hesaplama yoğun olabilir.*

In [ ]:
# Basit Sen's Slope Örneği (NDVI üzerinden)
def add_time_band(img):
    return img.addBands(img.metadata('year').rename('t'))

ts_with_time = time_series.map(add_time_band).select(['t', 'NDVI'])

# Sen's Slope Hesabı
slope = ts_with_time.reduce(ee.Reducer.sensSlope())

# Görselleştirme parametreleri
slope_vis = {
    'min': -0.05,
    'max': 0.05,
    'palette': ['red', 'white', 'green']
}

# Harita çıktısı (Folium veya URL) buraya eklenebilir
print("Trend analizi tanımlandı. .getInfo() veya .getMapId() ile sonuç alınabilir.")

## 4. Kayıp Nedenlerinin Ayrıştırılması (Taslak)
Bu aşamada CORINE Land Cover, Maden Ruhsat Alanları ve Yangın verileri kullanılarak değişimlerin nedeni belirlenir.

**Algoritma:**
1. `Değişim` = 1 (Eğer Sen's Slope < Eşik ise)
2. `Yangın` = 1 (Eğer dNBR > Yangın Eşiği ve Tarihsel Yangın Kaydı varsa)
3. `Maden` = 1 (Eğer Maden Ruhsat Sahası içindeyse)
4. `Kesim` = 1 (Diğer orman kayıpları)

*Not: Harici veriler (MTA Shapefile vb.) GEE'ye asset olarak yüklenmelidir.*